In [48]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import requests
import base64
import sqlite3 as db
sys.path.append('../')

pd.options.display.max_rows = 99

In [52]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [53]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning('Unable to connect to database due to: {}'.format(cnxError))
    return cnx

In [54]:
cnx = connectDB()
cur = cnx.cursor()

In [3]:
def foo(x):
    return x + 1

In [55]:
def getTrailheadRef():
    TrailheadRef = pd.DataFrame()
    try:
        trailheadRef = pd.read_sql('select * from trailheadRef;',con=cnx)
        trailheadRef.set_index('index',inplace=True)
        trailheadRef.convert_dtypes()
    except db.DatabaseError as dbError:
        print(dbError)
    return trailheadRef

In [10]:
trailheadRef = getTrailheadRef()

In [49]:
##  For the sake of demonstration please assume the data in the 'trailheadRef' comes from a King County Metro service
##  and/or is user-provided so some cleaning is required
def clean_trailheadRef(Latitude: float,Longitude: float, Address: str):
    trailheads = getTrailheadRef()
    ##set types for error checking, other cleaning
    trailheads.Latitude = trailheads['Latitude'].fillna(0.00).astype('float64',errors='ignore')
    trailheads.Longitude = trailheads['Longitude'].fillna(0.00).astype('float64',errors='ignore')
    trailheads.Address = trailheads['Address'].astype(str)

    updateMask = trailheads.apply(lambda x: bool(x.Address.strip()) and (x.Latitude == 0.00) and (x.Longitude == 0.00),axis=1)
    ##write stops that fail the updateMask checks to the db as logs
    try:
        trailheads[updateMask].copy().to_sql('geo_errorLogs',con=cnx,if_exists='append')
    except db.DatabaseError as dbError:
        cur.execute('INSERT INTO sql_errorLogs VALUES (?, ?)',dbError,str(dt.date.today()))
    return trailheads[~updateMask]

    ##needsGeo = trailheads[updateMask].copy()
    ##TODO: #25 fetch coordinates of trailheads in the update mask using Nominatim address service
    ##TODO #48 merge or call results of clean_trailheadRef() with dropCases()
#def dropCases():
    #dropCase = clean_trailheadRef()
    #dropMask = dropCase.apply(lambda y: (y.Address == '') | (y.Address == 'None'),axis=1)
    #noGeo = [y for y in list(dropCase[dropMask].StopName) if y!='']
    #if noGeo:
        #raise Exception('unable to fetch coordinates for {",".join(noGeo)}')
    #return dropCase[~dropMask]
    ##if there is no address or coordinates, drop the line


In [56]:
##it's working?
geoGeo = clean_trailheadRef(trailheadRef.Latitude,trailheadRef.Longitude,trailheadRef.Address)

Trailhead eBird hotspot update

1. Ask the eBird API for the latest list of hotspots for each trailhead
2. Add new eBird hotspots to the table 'Hotspots' in the trailheadDirectBirds_sous database
3. Update hotspot data